In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import seaborn as sns
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
!pip install jdatetime
import jdatetime

***
Train CSV

In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head()

,date,sale
0,1399-05-30,384.0
1,1399-05-31,393.0
2,1399-06-01,414.0
3,1399-06-02,410.0
4,1399-06-03,398.0


In [ ]:
df_train = pd.concat([df_train['date'].str.split('-', expand=True), 
                df_train['sale']], 
               axis=1)

df_train.columns = ['year', 'month', 'day', 'sale']
df_train.head()

,year,month,day,sale
0,1399,05,30,384.0
1,1399,05,31,393.0
2,1399,06,01,414.0
3,1399,06,02,410.0
4,1399,06,03,398.0


In [ ]:
df_train['date'] = ''
for i in range(0,len(df_train)): 
  df_train['date'][i] = jdatetime.date(int(df_train['year'][i]),
                                 int(df_train['month'][i]),
                                 int(df_train['day'][i])
                                 ).togregorian()
df_train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,year,month,day,sale,date
0,1399,05,30,384.0,2020-08-20
1,1399,05,31,393.0,2020-08-21
2,1399,06,01,414.0,2020-08-22
3,1399,06,02,410.0,2020-08-23
4,1399,06,03,398.0,2020-08-24


In [ ]:
df_train = df_train[['date', 'sale']]
df_train.head()

,date,sale
0,2020-08-20,384.0
1,2020-08-21,393.0
2,2020-08-22,414.0
3,2020-08-23,410.0
4,2020-08-24,398.0


***
Test CSV

In [ ]:
df_test = pd.read_csv('output.csv')
df_test.head()

,date,sale
0,1399-09-22,NaN
1,1399-09-23,NaN
2,1399-09-24,NaN
3,1399-09-25,NaN
4,1399-09-26,NaN


In [ ]:
df_test = pd.concat([df_test['date'].str.split('-', expand=True), 
                     df_test['sale']], 
                    axis=1)
df_test.columns = ['year', 'month', 'day', 'sale']
df_test.head()

,year,month,day,sale
0,1399,09,22,NaN
1,1399,09,23,NaN
2,1399,09,24,NaN
3,1399,09,25,NaN
4,1399,09,26,NaN


In [ ]:
df_test['date'] = ''
for i in range(0,len(df_test)): 
  df_test['date'][i] = jdatetime.date(int(df_test['year'][i]),
                                 int(df_test['month'][i]),
                                 int(df_test['day'][i])
                                 ).togregorian()
df_test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,year,month,day,sale,date
0,1399,09,22,NaN,2020-12-12
1,1399,09,23,NaN,2020-12-13
2,1399,09,24,NaN,2020-12-14
3,1399,09,25,NaN,2020-12-15
4,1399,09,26,NaN,2020-12-16


In [ ]:
df_test = df_test['date']
df_test.head()

0    2020-12-12
1    2020-12-13
2    2020-12-14
3    2020-12-15
4    2020-12-16
Name: date, dtype: object

***
model

In [ ]:
train = pd.DataFrame()
train['ds'] = df_train['date']
train['y'] = df_train['sale'] 
train.tail()

,ds,y
109,2020-12-07,262.0
110,2020-12-08,306.0
111,2020-12-09,292.0
112,2020-12-10,307.0
113,2020-12-11,379.0


In [ ]:
model = Prophet()

model.fit(train)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
sales = model.predict(df=df_test.reset_index().rename(columns={'date':'ds'}))
sales = round(sales['trend'])
sales.head()

0    368.0
1    367.0
2    366.0
3    365.0
4    364.0
Name: trend, dtype: float64

In [ ]:
df_test = pd.read_csv('output.csv')
df_test.head()

,date,sale
0,1399-09-22,NaN
1,1399-09-23,NaN
2,1399-09-24,NaN
3,1399-09-25,NaN
4,1399-09-26,NaN


In [ ]:
df_test['sale'] = sales
df_test.head()

,date,sale
0,1399-09-22,368.0
1,1399-09-23,367.0
2,1399-09-24,366.0
3,1399-09-25,365.0
4,1399-09-26,364.0


In [ ]:
df_test.to_csv('output.csv', index=False)